In [1]:
import pandas as pd

In [2]:
data = pd.read_json('/home/higoreller/pos_projects/datasets/feed_news.json')

In [9]:
data.sample(2)

,category,body,subtitle,title,url,spider,date_edit,source,date_retrieved,keywords,date_published,author
796,G1 - Mundo,"Chegar à Casa Branca não exige apenas carisma,...",Saiba quem doa mais fundos aos presidenciáveis...,Saiba quem doa mais fundos aos presidenciáveis...,http://g1.globo.com/mundo/noticia/2012/08/saib...,globo_g1,2012-08-24T21:13:00,Da EFE,2013-07-30T01:08:07.833890,"notícias, notícia, Mundo, Últimas notícias",2012-08-24T20:30:00,NaN
1708,Economia,Impulsionadas pela arrecadação recorde de impo...,Superávit das contas do governo é recorde para...,Superávit das contas do governo é recorde para...,http://g1.globo.com/economia/noticia/2013/02/c...,globo_g1,2013-02-26T12:46:00,"Do G1, em Brasília",2013-07-30T01:10:24.162094,"notícias, notícia, Economia, Últimas notícias",2013-02-26T12:09:00,Alexandro Martello


In [ ]:
# Instalando/atualizando o spacy
# !pip install -U spacy > /dev/null

# Baixando o modelo adequado
!python -m spacy download pt_core_news_lg

In [6]:
import spacy

nlp = spacy.load("pt_core_news_lg")

In [8]:
from tqdm.auto import tqdm 
tqdm.pandas()

In [10]:
data['docs'] = data.body.progress_map(nlp)

100%|██████████| 3882/3882 [02:33<00:00, 25.26it/s]


In [ ]:
data.to_pickle("documents.pickle")

In [11]:
companies = []
for doc in data.docs:
    orgs = []
    for ent in doc.ents:
        if ent.label_ != "ORG":
            continue
        orgs.append(ent.text)
    orgs = list(set(orgs))
    if len(orgs) > 1:
        companies.append(orgs)

In [ ]:
companies[0:2]

In [16]:
import itertools
edges = []

for nodes in companies:
    edges.extend([[n1, n2] for n1, n2 in itertools.combinations(nodes, 2)])

edges = pd.DataFrame(edges, columns=['source', 'target'])
edges

,source,target
0,Sena-MA,RR
1,Sena-MA,Programa das Nações Unidas para o Desenvolvimento
2,Sena-MA,Pnud
3,RR,Programa das Nações Unidas para o Desenvolvimento
4,RR,Pnud
...,...,...
78873,Google,Apple
78874,Google,Samsung
78875,IDC,Apple
78876,IDC,Samsung


In [14]:
edges_weighted = edges.groupby(edges.columns.tolist(), as_index=False).size().sort_values(by="size", ascending=False)

In [18]:
edges_weighted.columns = ['source', 
                          'target',
                          'weight']
edges_weighted

,source,target,weight
3125,Anatel,Agência Nacional de Telecomunicações,104
6063,Banco Central,BC,97
3716,Apple,Samsung,95
24916,Google,Apple,86
3237,Anatel,Oi,79
...,...,...,...
18972,FAA,Boeing,1
18973,FAA,G1 Economia,1
18974,FAB,Air Costa,1
18975,FAB,Air Lituanica,1


In [17]:
import networkx as nx

graph = nx.from_pandas_edgelist(edges_weighted)

graph

nx.write_graphml(graph, 'companies.graphml')